In [1]:
import os
import json
import time
import copy
from copy import deepcopy
from collections import defaultdict

import numpy as np
import math
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models

from skimage import io

import matplotlib.pyplot as plt
from matplotlib import patches, patheffects

import imgaug as ia
from imgaug import augmenters as iaa

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

from tqdm import tqdm
from pprint import pprint

In [2]:
# from torchsummary import summary

In [3]:
# base_path = r'../input'
base_path = r'input'
PATH_TRAIN_ANNO = os.path.join(base_path, 'train.csv')
PATH_TRAIN_IMG = os.path.join(base_path, 'train')

In [4]:
try:
    from tensorboardX import SummaryWriter
    USE_TENSORBOARD = True
    writer = SummaryWriter()
except:
    USE_TENSORBOARD = False
    print('No tensorboard X')

def record_tb(phase, tag, value, global_step):
    if USE_TENSORBOARD is True:
        writer.add_scalar('{phase}/{tag}'.format(phase=phase, tag=tag), value, global_step)

In [5]:
# os.listdir(PATH_TRAIN_IMG)[:10]

In [6]:
NUM_CLASSES = 28
MAX_TAGS = 5
IMG_SIZE = 224
BATCH_SIZE = 64
VAL_SIZE =0.2
THRESHOLD = 0.5
SAMPLES = 1
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
NUM_WORKERS = 4

In [7]:
def get_transform_anno(annotation_path, img_path):
    df = pd.read_csv(annotation_path)
    annotations = []
    for i, row in df.iterrows():
        rcd_id = row['Id']
        rcd_cate =  [int(j) for j in row['Target'].split()]
        annotations.append((rcd_id, rcd_cate))
    return annotations
#get annotations
annotations = get_transform_anno(PATH_TRAIN_ANNO, PATH_TRAIN_IMG)
sample_size = int(len(annotations) * SAMPLES)
print('sample size: {}'.format(sample_size))
annotations = annotations[:sample_size]
pprint(annotations[:3])

sample size: 31072
[('00070df0-bbc3-11e8-b2bc-ac1f6b6435d0', [16, 0]),
 ('000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0', [7, 1, 2, 0]),
 ('000a9596-bbc4-11e8-b2bc-ac1f6b6435d0', [5])]


In [8]:
class ProteinDataset(Dataset):
    def __init__(self, img_meta, img_path, transform = None):
        self.img_meta = img_meta
        self.transform = transform
        self.channels = ['red', 'blue', 'yellow', 'green']
        self.img_path = img_path
        self.mlb = MultiLabelBinarizer(classes=range(0,NUM_CLASSES))
        
    def __len__(self):
        return len(self.img_meta)

    def __getitem__(self, idx):
        img_id, img_tags= self.img_meta[idx]
        ch = []
        img_file_template = '{}_{}.png'
        for c in self.channels:
            ch.append(io.imread(os.path.join(self.img_path, img_file_template.format(img_id, c))))
        img = np.stack(ch)

        #augmentation
        if bool(self.transform) is True:
            img = self.transform(img)
            
        #binarize
        img_tags = self.mlb.fit_transform([img_tags]).squeeze()
        
        #transform to tensor
        img = torch.from_numpy(img).float()
        img_tags = torch.from_numpy(img_tags)
        
        output = (img, img_tags)
        return output

In [9]:
class ImgTfm:
    def __init__(self, aug_pipline = None):
        self.seq = aug_pipline
    
    def __call__(self, img):
        
#         seq_det = self.seq.to_deterministic()
        
        #augmentation
        aug_img=img.copy().transpose((1, 2, 0))
        aug_img = self.seq.augment_images([aug_img])[0]
        aug_img=aug_img.transpose((2, 1, 0))
        
        #normalize
        aug_img=aug_img/255
        
        return aug_img

In [10]:
def get_aug_pipline(img_size, mode = 'train'):
    if mode == 'train':
        seq = iaa.Sequential([
            iaa.Scale({"height": IMG_SIZE, "width": IMG_SIZE}),
            iaa.Sequential([
                iaa.Fliplr(0.5),
                iaa.Affine(
                    rotate=(-20, 20),
                )
            ], random_order=True) # apply augmenters in random order
        ], random_order=False)
    else: #ie.val
        seq = iaa.Sequential([
            iaa.Scale({"height": IMG_SIZE, "width": IMG_SIZE}),
        ], random_order=False)
#     seq = iaa.Sequential([
#                 iaa.Scale({"height": IMG_SIZE, "width": IMG_SIZE}),
#             ], random_order=False)
    return seq

In [11]:
train_set, val_set = train_test_split(annotations, test_size=VAL_SIZE, random_state=42)

composed = {}
composed['train'] = transforms.Compose([ImgTfm(aug_pipline=get_aug_pipline(img_size=IMG_SIZE, mode = 'train'))])
composed['val'] = transforms.Compose([ImgTfm(aug_pipline=get_aug_pipline(img_size=IMG_SIZE, mode = 'val'))])

image_datasets = {'train': ProteinDataset(train_set, img_path = PATH_TRAIN_IMG, transform=composed['train']),
                 'val': ProteinDataset(val_set, img_path = PATH_TRAIN_IMG, transform=composed['val'])}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, drop_last=True)
              for x in ['train', 'val']}

In [12]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print(dataset_sizes)

{'train': 24857, 'val': 6215}


In [13]:
#test dataset
# ix = 10
# tmp_img, tmp_tags  = image_datasets['train'][ix]

In [14]:
#test dataloader
# tmp_img, tmp_tags = next(iter(dataloaders['train']))
# print('tmp_img shape: {}\ntmp_tags: shape {}'.format(tmp_img.shape, tmp_tags.shape))

In [15]:
class Flatten(nn.Module):
    def __init__(self): 
        super().__init__()
    def forward(self, x): 
        return x.view(x.size(0), -1)

class RnetBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = self._prep_backbone()
        
    def _prep_backbone(self):     
        base_model = models.resnet34(pretrained=True)
        removed = list(base_model.children())[1:-2]
        backbone = nn.Sequential(*removed)
#         for param in backbone.parameters():
#             param.require_grad = False
        return backbone
    
    def forward(self, x):
        x = self.backbone(x)
        return x

class CustomHead(nn.Module):
    def __init__(self, num_class):
        super().__init__()
        self.num_class = num_class
        
        self.flatten = Flatten()
        self.relu_1 = nn.ReLU()
        self.dropout_1 = nn.Dropout(p=0.5)
        self.fc_2 = nn.Linear(512 * 7 * 7, 256)
        self.relu_2 = nn.ReLU()
        self.batchnorm_2 = nn.BatchNorm1d(256)
        self.dropout_2 = nn.Dropout(p=0.5)
        self.fc_3 = nn.Linear(256, self.num_class)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu_1(x)
        x = self.dropout_1(x)
        x = self.fc_2(x)
        x = self.relu_2(x)
        x = self.batchnorm_2(x)
        x = self.dropout_2(x)
        x = self.fc_3(x)
        return x

# class CustomEntry(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv_1 = nn.Conv2d(in_channels=4, out_channels=64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#         nn.init.kaiming_normal_(self.conv_1.weight, mode='fan_out', nonlinearity='relu')
        
#     def forward(self, x):
#         x = self.conv_1(x)
#         return x

class CustomEntry(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_1 = self._prep_layers()
        
    def _prep_layers(self):
        model = models.resnet34(pretrained=True)
        original_entry_w = torch.tensor(list(model.children())[0].weight)
        new_entry_w = torch.cat([original_entry_w, torch.zeros(size = (64,1,7,7))], 1)
        
        conv_1 = nn.Conv2d(in_channels=4, out_channels=64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        conv_1.weight=conv_1.weight = torch.nn.Parameter(new_entry_w)
        return conv_1
    
    def forward(self, x):
        x = self.conv_1(x)
        return x
    
class CustomNet(nn.Module):
    def __init__(self, num_class):
        super().__init__()
        self.custom_entry = CustomEntry()
        self.backbone = RnetBackbone()
        self.custom_head = CustomHead(num_class)
        
    def forward(self, x):
        x = self.custom_entry(x)
        x = self.backbone(x)
        x = self.custom_head(x)
        return x

In [16]:
class F1Loss(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, y_pred, y_true):
        #f1 loss
#         #prep y_true
        y_true = y_true.float()

        #prep y_pred
        y_pred = torch.tensor(data = (torch.sigmoid(y_pred).ge(THRESHOLD)), dtype=torch.float, device=DEVICE, requires_grad=True)

        #calculate loss
        tp = (y_true * y_pred).sum(0).float()
        # tn = ((1-y_true) * (1-y_pred)).sum(0).float()
        fp = ((1-y_true) * y_pred).sum(0).float()
        fn = (y_true * (1-y_pred)).sum(0).float()

        p = tp / (tp + fp)
        r = tp / (tp + fn)

        f1 = 2*p*r / (p+r)
        f1[torch.isnan(f1)] = 0
        f1_loss = 1-f1.mean()
#         print(f1_loss)
        return f1_loss

In [17]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, input, target):
        target = target.float()
        
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))

        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()

        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        
        return loss.sum(dim=1).mean()

In [18]:
def prep_stats(y_pred, y_true):
    #prep y_true
    y_true_tfm = y_true.cpu().numpy().astype('uint8')
    
    #prep y_pred khot
    y_pred_tfm = (torch.sigmoid(y_pred) > THRESHOLD).cpu().numpy().astype('uint8')
    
    return y_pred_tfm, y_true_tfm

In [19]:
def calc_stats(y_pred, y_true, stats = 'accurancy'):
    if stats == 'accuracy':
        stat_value = accuracy_score(y_true, y_pred)
    elif stats == 'precision':
        stat_value = precision_score(y_true, y_pred, average = 'macro')
    elif stats == 'recall':
        stat_value = recall_score(y_true, y_pred, average = 'macro')
    elif stats == 'f1':
        stat_value = f1_score(y_true, y_pred, average = 'macro')
    else:
        stat_value = 0
    return stat_value

In [20]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5, init_steps = 0):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_f1 = 0.0
    steps = init_steps

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            running_y_true = []
            running_y_pred = []
            
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            for inputs, targets in dataloaders[phase]:
                inputs = inputs.to(DEVICE)
                targets= targets.to(DEVICE)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    
                    y_pred_tfm, y_true_tfm = prep_stats(outputs, targets)
                    running_y_pred.append(y_pred_tfm)
                    running_y_true.append(y_true_tfm)
                    
                    #export step stats duing training phase
                    if phase == 'train':
                        record_tb(phase, 'loss', loss.cpu().data.numpy(), steps)
                        record_tb(phase, 'accuracy', calc_stats(y_pred_tfm, y_true_tfm, stats = 'accurancy'), steps)
                        record_tb(phase, 'precision', calc_stats(y_pred_tfm, y_true_tfm, stats = 'precision'), steps)
                        record_tb(phase, 'recall', calc_stats(y_pred_tfm, y_true_tfm, stats = 'recall'), steps)
                        record_tb(phase, 'f1', calc_stats(y_pred_tfm, y_true_tfm, stats = 'f1'), steps)
                        steps += 1
                        
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)

            #calc epoch stats
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = accuracy_score(np.vstack(running_y_true), np.vstack(running_y_pred))
            epoch_precision = precision_score(np.vstack(running_y_true), np.vstack(running_y_pred), average = 'macro')
            epoch_recall = recall_score(np.vstack(running_y_true), np.vstack(running_y_pred), average = 'macro')
            epoch_f1 = f1_score(np.vstack(running_y_true), np.vstack(running_y_pred), average = 'macro')
            
            #export epoch stats duing training phase
            if phase == 'val':
                record_tb(phase, 'loss', epoch_loss, steps)
                record_tb(phase, 'accuracy', epoch_acc, steps)
                record_tb(phase, 'precision', epoch_precision, steps)
                record_tb(phase, 'recall', epoch_recall, steps)
                record_tb(phase, 'f1', epoch_f1, steps)
            
            print('{} Loss: {:.4f} Acc: {:.4f} Percision: {:.4f} Recall {:.4f} F1 {:.4f}'.format(
                phase, epoch_loss, epoch_acc, epoch_precision, epoch_recall, epoch_f1))

            # deep copy the model
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
#     print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
#     model.load_state_dict(best_model_wts)
    return model, steps

In [21]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = CustomNet(num_class=NUM_CLASSES)
model_ft = model_ft.to(DEVICE)

criterion = FocalLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.01)

# Decay LR by a factor of 0.1 every 10 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [22]:
#Freeze backbone
for param in model_ft.backbone.parameters():
    param.requires_grad = False

In [23]:
steps = 0

In [24]:
model_ft, steps = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20, init_steps=steps)

Epoch 0/19
----------


/home/spacor/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/spacor/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/spacor/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/spacor/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


train Loss: 1.3525 Acc: 0.0561 Percision: 0.0887 Recall 0.0380 F1 0.0473
val Loss: 1.2339 Acc: 0.0759 Percision: 0.0896 Recall 0.0393 F1 0.0423

Epoch 1/19
----------
train Loss: 1.1881 Acc: 0.0795 Percision: 0.2713 Recall 0.0464 F1 0.0587
val Loss: 1.4990 Acc: 0.0986 Percision: 0.2027 Recall 0.0509 F1 0.0647

Epoch 2/19
----------
train Loss: 1.1561 Acc: 0.0935 Percision: 0.2585 Recall 0.0567 F1 0.0749
val Loss: 1.1557 Acc: 0.1211 Percision: 0.1260 Recall 0.0647 F1 0.0745

Epoch 3/19
----------
train Loss: 1.1334 Acc: 0.1031 Percision: 0.2777 Recall 0.0661 F1 0.0891
val Loss: 1.0991 Acc: 0.0994 Percision: 0.3099 Recall 0.0608 F1 0.0874

Epoch 4/19
----------
train Loss: 1.1197 Acc: 0.1084 Percision: 0.3074 Recall 0.0727 F1 0.1003
val Loss: 1.0956 Acc: 0.1200 Percision: 0.2855 Recall 0.0680 F1 0.0896

Epoch 5/19
----------
train Loss: 1.1065 Acc: 0.1186 Percision: 0.3087 Recall 0.0789 F1 0.1087
val Loss: 1.1034 Acc: 0.1361 Percision: 0.3920 Recall 0.0805 F1 0.1122

Epoch 6/19
---------

In [25]:
#save intermediate model
torch.save(model_ft.state_dict(), 'intermediate_11042018.model')

In [26]:
#freeze entry
for param in model_ft.custom_entry.parameters():
    param.requires_grad = False
#unfreeze last covn of backbnoe
for param in list(list(model_ft.backbone.children())[0][-1].parameters()):
    param.requires_grad = True

In [27]:
# Different learning rate for different layers
# optimizer_ft = optim.Adam([
#     {'params': model_ft.custom_entry.parameters()},
#     {'params': model_ft.backbone.parameters(), 'lr': 0.001},
#     {'params': model_ft.custom_head.parameters()},
#     ]
# )
# # Decay LR by a factor of 0.1 every n epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [28]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=50, init_steps=steps)

Epoch 0/49
----------


/home/spacor/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/spacor/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/spacor/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/spacor/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


train Loss: 1.1198 Acc: 0.1252 Percision: 0.3084 Recall 0.0797 F1 0.1084
val Loss: 1.0514 Acc: 0.1619 Percision: 0.3100 Recall 0.0901 F1 0.1173

Epoch 1/49
----------
train Loss: 1.0447 Acc: 0.1737 Percision: 0.3490 Recall 0.1186 F1 0.1588
val Loss: 1.0830 Acc: 0.2049 Percision: 0.2898 Recall 0.1394 F1 0.1784

Epoch 2/49
----------
train Loss: 1.0186 Acc: 0.1860 Percision: 0.4055 Recall 0.1346 F1 0.1796
val Loss: 1.0009 Acc: 0.2020 Percision: 0.3355 Recall 0.1333 F1 0.1718

Epoch 3/49
----------
train Loss: 1.0026 Acc: 0.1955 Percision: 0.3811 Recall 0.1443 F1 0.1909
val Loss: 0.9670 Acc: 0.2287 Percision: 0.4086 Recall 0.1551 F1 0.2000

Epoch 4/49
----------
train Loss: 0.9870 Acc: 0.2052 Percision: 0.3937 Recall 0.1509 F1 0.1991
val Loss: 1.0586 Acc: 0.2205 Percision: 0.3742 Recall 0.1539 F1 0.2010

Epoch 5/49
----------
train Loss: 0.9751 Acc: 0.2085 Percision: 0.3897 Recall 0.1566 F1 0.2055
val Loss: 0.9586 Acc: 0.2165 Percision: 0.4044 Recall 0.1561 F1 0.2033

Epoch 6/49
---------